# K ta yaqin qo'shni usuli

Ushbu qismda o'rganmoqchi bo'lgan algoritmimiz bu *k ta yaqin qo'shni* deb nomlanadi. Ushbu usul biz rasmlardagi raqamlarni aniqlashda ishlatilgan usulning umumiyroq ko'rinishi hisoblanadi. Hamda bu usulni ham biz umumiy holda "Bayes" usuli deyishimiz mumkin. Chunki ushbu usul ham oldingi mavjud to'plamdan foydalanib obyektga baho beradi. *Yaqin qo'shni* usuli asosida rasmidagi raqamni topayotganimizda bitta yaqin qo'shinisi qaragan edik. Endi shuni kengaytirgan holda bitta emas *k* ta qo'shni e'tiborga olamiz. Undan tashqari ushbu bobda biz ushbu usulni umumlashtirib regressiya masalalari uchun ham qo'llaymiz.

## 

In [2]:
import numpy as np

In [3]:
data = np.loadtxt('/home/tqqt1/AI/teachings/online-courses/ai_intro/datasets/Dry_Bean_Dataset.csv',
           skiprows=1,
           delimiter=',')
X = data[:, :-1]
y = data[:, -1].astype(int)

In [4]:
num_train = 10_000
num_val = 3_000

num_test = 611

In [5]:
indices = np.arange(data.shape[0])
np.random.seed(42)
np.random.shuffle(indices)

X_tr = X[indices[:num_train]]
y_tr = y[indices[:num_train]]

X_va = X[indices[num_train:num_train+num_val]]
y_va = y[indices[num_train:num_train+num_val]]

X_te = X[indices[num_train+num_val:]]
y_te = y[indices[num_train+num_val:]]

In [6]:
x_tr_max = X_tr.max(axis=0)

X_tr = X_tr / x_tr_max
X_va = X_va / x_tr_max
X_te = X_te / x_tr_max

In [31]:
D = np.zeros((num_val, num_train))
# val to'plam bo'yicha loop
i = 0
while i < num_val:
    # train to'plam bo'yicha loop
    j = 0
    while j < num_train:
        # Manhattan
        # D[i, j] = np.sum(np.abs(X_va[i] - X_tr[j]))
        # Euclidean
        D[i, j] = np.sqrt(np.sum((X_va[i] - X_tr[j]) ** 2))
        j += 1 # j = j + 1
    i += 1 # i = i + 1
D_argsort = np.argsort(D, axis=1)

In [33]:
k_best = 0
acc_best = 0

k = 2

while k < 100:
    y_pred = np.zeros_like(y_va)
    i = 0
    while i < num_val:
        uniq_labels, label_counts = np.unique(y_tr[D_argsort[i, :k]], return_counts=True)
        y_pred[i] = uniq_labels[np.argmax(label_counts)]
        i += 1
    A = np.mean(y_pred == y_va)
    # print(f"k={k} da aniqlik: {A:.4f}")
    if A > acc_best:
        acc_best = A
        k_best = k
    k += 1 # k = k + 1

print(f"Eng yaxshi aniqlik {acc_best:.4f} {k_best} ta yaqin qo'shnida...")

Eng yaxshi aniqlik 0.9150 39 ta yaqin qo'shnida...


In [34]:
D = np.zeros((num_test, num_train))
# val to'plam bo'yicha loop
i = 0
while i < num_test:
    # train to'plam bo'yicha loop
    j = 0
    while j < num_train:
        # D[i, j] = np.sum(np.abs(X_te[i] - X_tr[j]))
        D[i, j] = np.sqrt(np.sum((X_te[i] - X_tr[j]) ** 2))
        j += 1 # j = j + 1
    i += 1 # i = i + 1
D_argsort = np.argsort(D, axis=1)

In [35]:
y_pred = np.zeros_like(y_te)
i = 0
while i < num_test:
    uniq_labels, label_counts = np.unique(y_tr[D_argsort[i, :k_best]], return_counts=True)
    y_pred[i] = uniq_labels[np.argmax(label_counts)]
    i += 1
A = np.mean(y_pred == y_te)

print(f"Testning aniqlik {A:.4f} {k_best} ta yaqin qo'shnida...")

Testning aniqlik 0.9034 39 ta yaqin qo'shnida...


Testning aniqlik 0.9083469721767594 7 ta yaqin qo'shnida...
Testning aniqlik 0.911620294599018 39 ta yaqin qo'shnida...


1. Underfitting - kam o'rganish
2. Just (Well) fitting - yaxshi o'rganish
3. Overfitting - Me'yoridan ortiq o'rganish

1. Manhattan

$p(a,b)=\sum_{i=1}^{n}|a_i-b_i|$

2. Euclidean

$p(a,b)=\sqrt{\sum_{i=1}^{n}(a_i-b_i)^2}$

4. Minkowski

$p(a,b)=(\sum_{i=1}^{n}(a_i-b_i)^p)^{\frac{1}{p}}$

## Vaznlashgan Knn

In [36]:
D = np.zeros((num_val, num_train))
# val to'plam bo'yicha loop
i = 0
while i < num_val:
    # train to'plam bo'yicha loop
    j = 0
    while j < num_train:
        # Manhattan
        # D[i, j] = np.sum(np.abs(X_va[i] - X_tr[j]))
        # Euclidean
        D[i, j] = np.sqrt(np.sum((X_va[i] - X_tr[j]) ** 2))
        j += 1 # j = j + 1
    i += 1 # i = i + 1
D_argsort = np.argsort(D, axis=1)

In [37]:
k_best = 0
acc_best = 0

k = 2

while k < 100:
    y_pred = np.zeros_like(y_va)
    i = 0
    while i < num_val:
        y_k = y_tr[D_argsort[i, :k]]
        
        D_k = D[i, :k]
        D_k_sum = np.sum(D_k)
        D_k_1 = D_k / D_k_sum
        D_k_2 = 1 - D_k_1
        w = D_k_2 / (k - 1)

        probs = np.zeros(y_tr.max()+1)
        j = 0
        while j < y_tr.max()+1:
            probs[j] = np.sum(w[y_k == j])
            j += 1

        y_pred[i] = np.argmax(probs)
        
        
        i += 1
    A = np.mean(y_pred == y_va)
    # print(f"k={k} da aniqlik: {A:.4f}")
    if A > acc_best:
        acc_best = A
        k_best = k
    k += 1 # k = k + 1

print(f"Eng yaxshi aniqlik {acc_best:.4f} {k_best} ta yaqin qo'shnida...")

Eng yaxshi aniqlik 0.9153 40 ta yaqin qo'shnida...


In [38]:
D = np.zeros((num_test, num_train))
# val to'plam bo'yicha loop
i = 0
while i < num_test:
    # train to'plam bo'yicha loop
    j = 0
    while j < num_train:
        # D[i, j] = np.sum(np.abs(X_te[i] - X_tr[j]))
        D[i, j] = np.sqrt(np.sum((X_te[i] - X_tr[j]) ** 2))
        j += 1 # j = j + 1
    i += 1 # i = i + 1
D_argsort = np.argsort(D, axis=1)

In [40]:
y_pred = np.zeros_like(y_te)
i = 0
while i < num_test:
    y_k = y_tr[D_argsort[i, :k]]
    
    D_k = D[i, :k]
    D_k_sum = np.sum(D_k)
    D_k_1 = D_k / D_k_sum
    D_k_2 = 1 - D_k_1
    w = D_k_2 / (k - 1)

    probs = np.zeros(y_tr.max()+1)
    j = 0
    while j < y_tr.max()+1:
        probs[j] = np.sum(w[y_k == j])
        j += 1

    y_pred[i] = np.argmax(probs)
    
    i += 1
A = np.mean(y_pred == y_te)

print(f"Testning aniqlik {A:.4f} {k_best} ta yaqin qo'shnida...")

Testning aniqlik 0.9002 40 ta yaqin qo'shnida...
